In [ ]:
import os

import numpy as np
from keras.optimizers import Adam
from keras.callbacks import Callback, LearningRateScheduler
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
from PIL import Image
import keras.backend as K

from unet import get_unet, jaccard_coef_int, jaccard_coef_loss

## load the model

In [ ]:
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [ ]:
model = get_unet(3, 512, 512, classes=1)

In [ ]:
def binary_error(y_true, y_pred):
    return K.mean(K.not_equal(y_true, K.round(y_pred)), axis=-1)

In [ ]:
adam = Adam(lr=1e-3)
model.compile(adam, loss=jaccard_coef_loss, metrics=['binary_crossentropy', jaccard_coef_int, binary_error])

# Explore labels load

In [ ]:
from pycocotools.mask import decode
import json
import matplotlib.pyplot as plt

In [ ]:
labels = json.load(open('/root/data/blender_v4/training/training_low_rez/labels.json'))

In [ ]:
Image.open(labels[0]['path'])

In [ ]:
sum_mask = np.sum(decode(labels[0]['masks']), axis=2)
sum_mask[sum_mask>0]=1
plt.imshow(sum_mask)

## load data and define generators

In [ ]:
import glob
import cv2
import numpy as np
import random
import math
import json
SEED = 448
import matplotlib.pyplot as plt

In [ ]:
train_labels = json.load(open('/root/data/blender_v4/training/training_low_rez/labels.json'))
test_labels = json.load(open('/root/data/blender_v4/training/test_low_rez/labels.json'))

In [ ]:
# parameters 
batch_size = 8
steps_per_epoch = len(train_labels) // batch_size
steps_per_epoch_val = len(test_labels) // batch_size
input_shape = (512, 512, 3)

In [ ]:
def generator(labels, steps_per_epoch, BATCH_SIZE, input_shape):
    i = 0
    
    while True:
        x_batch = np.empty((BATCH_SIZE, input_shape[0], input_shape[1], input_shape[2]), dtype=np.uint8)
        y_batch = np.empty((BATCH_SIZE, input_shape[0], input_shape[1], 1), dtype=np.uint8)
        for (ind, j) in enumerate(range(i*BATCH_SIZE, (i+1)*BATCH_SIZE)):
            x_batch[ind,...] = np.array(Image.open(labels[j]['path']).resize((input_shape[0], input_shape[1])))
            sum_mask = np.sum(decode(labels[j]['masks']), axis=2)
            sum_mask[sum_mask>0]=1
            y_batch[ind,...,0] = cv2.resize(np.array(sum_mask, dtype=np.uint8), (input_shape[0], input_shape[1]))
        i += 1
        if i >= steps_per_epoch:
            i = 0
        yield x_batch, y_batch

In [ ]:
train_generator = generator(train_labels, steps_per_epoch, batch_size, input_shape)
val_generator = generator(test_labels, steps_per_epoch_val, batch_size, input_shape)

In [ ]:
X, Y = train_generator.next()

In [ ]:
np.array(Image.open(labels[0]['path'])).shape

In [ ]:
plt.imshow(X[0,...])
plt.imshow(Y[0,...].squeeze(), alpha=0.3)

## train

In [ ]:
# learning rate schedule
def step_decay(epoch):
    initial_lrate = 1e-3
    drop = 0.5
    epochs_drop = 10.0
    if epoch >= 30:
        fake_epoch = epoch - 20 
        lrate = initial_lrate * math.pow(drop, math.floor((1+fake_epoch)/epochs_drop))
    else:
        lrate = initial_lrate
    print('lr {}'.format(lrate))
    return lrate
lr_scheduler = LearningRateScheduler(step_decay)

In [ ]:
# create history callback
class SaveHistory(Callback):
    
    def __init__(self, json_path):
        self.json_path = json_path
    
    def on_train_begin(self, logs=None):
        self.epoch = []
        self.history = {}

    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        self.epoch.append(epoch)
        for k, v in logs.items():
            self.history.setdefault(k, []).append(v)
        with open(self.json_path, 'w') as f:
            json.dump(self.history, f)

In [ ]:
saveh = SaveHistory('./segmentation_history.json')

In [ ]:
# save model
filepath = os.path.join('/root/data/models/blender/segmentation/', 'segmentation_{epoch:02d}.h5')
checkpoint = ModelCheckpoint(filepath, 
                             monitor='val_jaccard_coef_int', 
                             save_best_only=True, 
                             mode='max')

In [ ]:
# start training# start 
history = model.fit_generator(
        generator=train_generator,
        steps_per_epoch=steps_per_epoch,
        epochs=50,
        verbose=1,
        callbacks=[saveh, lr_scheduler, checkpoint],
        validation_data= val_generator,
        validation_steps= steps_per_epoch_val)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
history = json.load(open('segmentation_history.json'))

In [ ]:
history.keys()

In [ ]:
np.max(history['val_jaccard_coef_int'])

In [ ]:
plt.plot(history['jaccard_coef_int'], label='jaccard_coef_int')
plt.plot(history['val_jaccard_coef_int'], label='val_jaccard_coef_int')
plt.legend()

In [ ]:
plt.plot(history['loss'], label='train_loss')
plt.plot(history['val_loss'], label='val_loss')
plt.legend()

# predict

In [ ]:
X, Y = val_generator.next()

In [ ]:
Ypred = model.predict_on_batch(X).squeeze()

In [ ]:
Ypred.shape

In [ ]:
for i in range(8):
    f, ax = plt.subplots(1, 2, figsize=(20, 10))
    ax[0].imshow(X[i,...])
    ax[0].imshow(Y[i,...,0], alpha=0.5)
    ax[0].set_title("background")
    
    ax[1].imshow(X[i,...])
    ax[1].imshow(Y[i,...,1], alpha=0.5)
    ax[1].set_title("foreground")
    plt.show()